In [1]:
import time
start_time = time.time()
best_epoch = 22
save_version = 'test_v1'
from dti_v3_data import *
from dti_v3_prep import *
from dti_v3_esoinn import *

[2021-12-14 06:24:18 DEBUG - MainProcess-MainThread - __init__.py(wrapper):275] matplotlib data path: /usr/local/lib/python3.7/site-packages/matplotlib/mpl-data
[2021-12-14 06:24:18 DEBUG - MainProcess-MainThread - __init__.py(wrapper):275] CONFIGDIR=/root/.config/matplotlib
[2021-12-14 06:24:18 DEBUG - MainProcess-MainThread - __init__.py(<module>):1445] interactive is False
[2021-12-14 06:24:18 DEBUG - MainProcess-MainThread - __init__.py(<module>):1446] platform is linux
[2021-12-14 06:24:18 DEBUG - MainProcess-MainThread - __init__.py(<module>):1447] loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_collections_abc', '_sitebuiltin

In [2]:
dataClass = DataCreation('pred')
strPrep = StrProcessing()
prep_data = strPrep.load_tfidf_model_trans(dataClass.str_data, list(dataClass.str_data), 10000, save_version)
X_pred = prep_data.values

PREDICT DATA START DATETIME:  2020-12-14 00:00:00
DATA END DATETIME:  2021-12-14 00:00:00
******* 예측 데이터 불러오기 *******
{'host': '192.168.0.42', 'port': '9001', 'timeout': '600000', 'thread': '10'}
DB client : <clickhouse_driver.client.Client object at 0x7f1431d74e50>
PREDICT DATA PROPERTIES
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276402 entries, 0 to 276401
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype                     
---  ------  --------------   -----                     
 0   lgtime  276402 non-null  datetime64[ns, Asia/Seoul]
 1   src_ip  276402 non-null  object                    
 2   dst_ip  276402 non-null  object                    
 3   host    276402 non-null  object                    
 4   agent   276402 non-null  object                    
 5   query   276402 non-null  object                    
 6   label   276402 non-null  object                    
dtypes: datetime64[ns, Asia/Seoul](1), object(6)
memory usage: 14.8+ MB
None
INDEXES :

In [3]:
print("X_pred.shape :", X_pred.shape)
print("정상 데이터 수 :" ,(dataClass.label_data == 'normal').sum())
print("비정상 데이터 수 :" ,(dataClass.label_data != 'normal').sum())

X_pred.shape : (276402, 322)
정상 데이터 수 : label    219912
dtype: int64
비정상 데이터 수 : label    56490
dtype: int64


In [4]:
s = ESoinn()
try:
    s.load_esoinn_model(save_version, best_epoch)
except Exception as er:
    print(er)
noise_idx_list, non_noise_list = s.predict(X_pred)

end_time = time.time() - start_time
print("::::::::::: %.2f seconds ::::::::::::::\n\n" % end_time)

esoinn_model load and initiated **************
::::::::::: 138.93 seconds ::::::::::::::




<Figure size 432x288 with 0 Axes>

In [ ]:
Y = dataClass.label_data['label']
idx_data = dataClass.index_data.values
pred_res_list = []
idx = 0
for label in Y:
    tmp_res_arr = list(idx_data[idx])
    pred = ""
    if idx in non_noise_list:
        pred += "normal"
    else:
        pred += "ATTACK"
    
    tmp_res_arr.append(label)
    tmp_res_arr.append(pred)
    tmp_res_arr.append(str(end_time))
    tmp_res_arr.append(str(best_epoch))
    tmp_res_arr.append(save_version)
    pred_res_list.append(tmp_res_arr)
    idx += 1

# execute_ch("INSERT INTO dti.kisa_ai_engine_performance_result VALUES", pred_res_list)

In [ ]:
with open("./output/performace_result.csv", "w") as f:
    f.write("logtime, src_ip, dst_ip, ai_label_true, ai_label_pred, performace(seconds), best_epoch, version\n")
    for res_arr in pred_res_list:
        tmp_line = ""
        for res_val in res_arr:
            tmp_line = tmp_line + str(res_val) + ","
        f.write(tmp_line[:-1] + "\n")

In [5]:
execute_ch("INSERT INTO dti.kisa_ai_engine_performance_result VALUES", pred_res_list)

{'host': '192.168.0.42', 'port': '9001', 'timeout': '600000', 'thread': '10'}
DB client : <clickhouse_driver.client.Client object at 0x7efb73b508d0>


In [7]:
cnt = 0
tot = 0
with open("./output/performace_result.csv", "r") as f:
    print(f.readline())
    while True:
        tmp_line = f.readline().strip()
        if tmp_line == '': break
        tmp_arr = tmp_line.split(',')
        tot += 1
        if tmp_arr[3] == tmp_arr[4]:
            cnt += 1
print(cnt, tot, cnt/tot)

logtime, src_ip, dst_ip, ai_label_true, ai_label_pred, performace(seconds), best_epoch, version

233107 276402 0.8433622043255837
